In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df = pd.read_csv("covid19.csv")

df

,data,casos
0,2020-03-02 18:00:00,2
1,2020-03-03 18:00:00,4
2,2020-03-04 18:00:00,6
3,2020-03-05 18:00:00,9
4,2020-03-06 18:00:00,13
5,2020-03-07 18:00:00,21
6,2020-03-08 18:00:00,30
7,2020-03-09 18:00:00,39
8,2020-03-10 18:00:00,41
9,2020-03-11 17:00:00,59


In [3]:
df = df.loc[:,['data','casos']]
FMT = '%Y-%m-%d %H:%M:%S'
date = df['data']
df['data'] = date.map(lambda x : (datetime.strptime(x, FMT) - datetime.strptime("2020-03-01 00:00:00", FMT)).days  )

In [4]:
def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))

In [7]:
x = list(df.iloc[:,0])
y = list(df.iloc[:,1])
fit = curve_fit(logistic_model,x,y,p0=[2,20,2000])

ValueError: could not convert string to float: '2020-03-02 18:00:00'

In [ ]:
fit


In [ ]:
errors = [np.sqrt(fit[1][i][i]) for i in [0,1,2]]

In [ ]:
errors

In [ ]:
a = fit[0][0]
b = fit[0][1]
c = fit[0][2]

In [ ]:
sol = int(fsolve(lambda x : logistic_model(x,a,b,c) - int(c),b))

In [ ]:
y_pred_logistic = [logistic_model(i,fit[0][0],fit[0][1],fit[0][2]) for i in x]

mean_squared_error(y,y_pred_logistic)


In [ ]:
#def exponential_model(x,a,b,c):
    #return a*np.exp(b*(x-c))
#exp_fit = curve_fit(exponential_model,x,y,p0=[2,20,2000])

In [ ]:
#exp_fit


In [ ]:
pred_x = list(range(max(x),sol))
plt.rcParams['figure.figsize'] = [7, 7]
plt.rc('font', size=14)
# Real data
plt.scatter(x,y,label="Dados Reais",color="red")
# Predicted logistic curve
plt.plot(x+pred_x, [logistic_model(i,fit[0][0],fit[0][1],fit[0][2]) for i in x+pred_x], label="Logistic model" )
plt.legend()
plt.xlabel("Dias desde 2 Março 2020")
plt.ylabel("Numero de infetados")
plt.ylim(0, c*1.5)
plt.title("Covid-19 Modelo Portugal")
plt.show()

In [ ]:
from datetime import timedelta, date
final = datetime.strptime("2020-03-02 00:00:00", FMT) + timedelta(days=sol)
final.strftime('End of pandemic predicted at: %d, %b %Y')